<a href="https://colab.research.google.com/github/lhaq/market-anomaly-detection/blob/main/Market_Anomaly_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install gradio -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.4/321.4 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.2 MB/s eta 0:00:00


In [ ]:
import gradio as gr
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score, precision_score, recall_score, f1_score, classification_report
import matplotlib.pyplot as plt

def analyze_financial_market_data(df):
    # Ensure the 'Data' column is in datetime format
    df['Data'] = pd.to_datetime(df['Data'])

    # Calculate the 28-day moving average of VIX
    df['VIX_28_MA'] = df['VIX'].rolling(window=4).mean()

    # Define the target variable for VIX above 30
    df['VIX_above_30'] = (df['VIX_28_MA'] > 30).astype(int)

    # Create features and target dataset for VIX prediction
    features_vix = df[['XAU BGNL', 'USGG2YR', 'MXUS', 'MXEU', 'MXJP', 'MXBR', 'MXRU', 'MXIN', 'MXCN']]
    target_vix = df['VIX_above_30']

    # Split the data into training and testing sets for VIX prediction
    X_train_vix, X_test_vix, y_train_vix, y_test_vix = train_test_split(features_vix, target_vix, test_size=0.2, random_state=42)

    # Train the XGBoost model for VIX prediction
    model_vix = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
    model_vix.fit(X_train_vix, y_train_vix)

    # Predict on the test set for VIX prediction
    y_pred_vix = model_vix.predict(X_test_vix)

    # Evaluate the model for VIX prediction
    mse_vix = mean_squared_error(y_test_vix, y_pred_vix)
    result = f'Mean Squared Error for VIX prediction: {mse_vix}\n'

    # Generate the classification report for VIX prediction
    report_vix = classification_report(y_test_vix, y_pred_vix)

    # Predict the 28-day moving average of VIX
    df['VIX_28_MA_Pred'] = model_vix.predict(features_vix)

    # Investment strategy
    def investment_strategy(row):
        if row['VIX_28_MA_Pred'] == 1:
            if row['VIX_28_MA'] > 30:
                return 'Convert assets to XAU'
            elif row['VIX'] > row['VIX_28_MA']:
                return 'Hold'
            elif row['VIX'] < row['VIX_28_MA']:
                return 'Convert some XAU to other assets'
        return 'No action'

    df['Investment_Strategy'] = df.apply(investment_strategy, axis=1)

    # Display the investment strategy
    result += df[['Data', 'VIX', 'VIX_28_MA', 'VIX_28_MA_Pred', 'Investment_Strategy']].to_string() + "\n"

    # Plot the VIX and the 28-day moving average
    plt.figure(figsize=(14, 7))
    plt.plot(df['Data'], df['VIX'], label='VIX')
    plt.plot(df['Data'], df['VIX_28_MA'], label='28-day MA of VIX')
    plt.xlabel('Date')
    plt.ylabel('VIX')
    plt.title('VIX and its 28-day Moving Average')
    plt.legend()
    plt.savefig('vix_plot.png')
    plt.close()

    # Define the target variable for anomaly prediction
    target_anomaly = df['Y']

    # Split the data into training and testing sets for anomaly prediction
    X_train_anomaly, X_test_anomaly, y_train_anomaly, y_test_anomaly = train_test_split(features_vix, target_anomaly, test_size=0.2, random_state=42)

    # Train the XGBoost model for anomaly prediction
    model_anomaly = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
    model_anomaly.fit(X_train_anomaly, y_train_anomaly)

    # Predict on the test set for anomaly prediction
    y_pred_anomaly = model_anomaly.predict(X_test_anomaly)

    # Evaluate the model for anomaly prediction
    accuracy = accuracy_score(y_test_anomaly, y_pred_anomaly)
    precision = precision_score(y_test_anomaly, y_pred_anomaly)
    recall = recall_score(y_test_anomaly, y_pred_anomaly)
    f1 = f1_score(y_test_anomaly, y_pred_anomaly)

    anomaly_metrics = (
        f'Accuracy for anomaly prediction: {accuracy}\n'
        f'Precision for anomaly prediction: {precision}\n'
        f'Recall for anomaly prediction: {recall}\n'
        f'F1 Score for anomaly prediction: {f1}\n'
    )

    # Generate the classification report for anomaly prediction
    report_anomaly = classification_report(y_test_anomaly, y_pred_anomaly)

    return result, 'vix_plot.png', report_vix, anomaly_metrics, report_anomaly

# Gradio UI
def gradio_interface(file):
    df = pd.read_csv(file.name, parse_dates=['Data'], date_parser=lambda x: pd.to_datetime(x, format='%d/%m/%Y'))
    return analyze_financial_market_data(df)

inputs = gr.File(label="Upload CSV file")
outputs = [
    gr.Textbox(label="Analysis Results"),
    gr.Image(label="VIX Plot"),
    gr.Textbox(label="VIX Classification Report"),
    gr.Textbox(label="Anomaly Metrics"),
    gr.Textbox(label="Anomaly Classification Report")
]

gr.Interface(fn=gradio_interface, inputs=inputs, outputs=outputs, title="Market Anomaly Detection").launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://edbfa7a0ac6975df8a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
